In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

![](https://static.wixstatic.com/media/72c0b2_3f82b019e67049739a9a13ecd693b770~mv2.png/v1/fill/w_1000,h_571,al_c,usm_0.66_1.00_0.01/72c0b2_3f82b019e67049739a9a13ecd693b770~mv2.png)

# GİRİŞ

Bu çalışmada, farklı tipte insanların sağlık harcalamaları kural tabanlı sınıflandırılıp tahmin edilmeye çalışılacaktır. Çalışmanın özeti şöyledir:

1. Veri Setinin Yüklenmesi
1. Veri Setine Genel Bakış
1. Veri Manipülasyonu
1. Persona Oluşturulması
1. Sonuç


## 1.Veri Setinin Yüklenmesi

In [ ]:
#Gerekli kütüphaneler ortama yükleniyor

import seaborn as sns
import matplotlib.pyplot as plt
#pd.set_option("display.max_rows",None)
pd.set_option("display.expand_frame_repr",False)
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#Veri seti yükleniyor

df=pd.read_csv("/kaggle/input/insurance/insurance.csv")

## 2.Veri Setine Genel Bakış

Veri setinde 7 değişken 1338 örnek var.
Bu değişkenler:
* age : yaş 
* sex : cinsiyet
* bmi : vücut kitle indexi
* children : sahip olduğu çocuk sayısı
* smoker : sigara içip içmediği
* region : bulunduğu bölge
* charges : ödenen ücret bilgisini içeriyor.

In [ ]:
def control_df(df):
    print("###############################################################")
    print("İlk 5 veri")
    print(df.head())
    print("###############################################################")
    print("Verinin boyutları:")
    print(df.shape)
    print("###############################################################")
    print("Verideki boş gözlem sayısı:")
    print(df.isnull().sum())
    cat_cols=[col for col in df.columns if df[col].dtype=="O"]
    num_but_cat_cols=[col for col in df.columns if df[col].nunique() <10 and df[col].dtype != "O"]
    cat_but_num_cols=[col for col in df.columns if df[col].nunique() > 20 and df[col].dtype == "O"]
    cat_cols = cat_cols + num_but_cat_cols
    num_cols = [col for col in df.columns if col not in cat_cols]
    print("###############################################################")
    print("Verideki kategorik değişkenler")
    print(cat_cols)
    print("###############################################################")
    print("Verideki sayısal değişkenler")
    print(num_cols)
    
    return cat_cols,num_cols 

In [ ]:
#Veri setinin genel özellikleri
#kategorik verilerin ve nümerik verilerin değişkenlere atanması
cat_cols,num_cols=control_df(df)

Kategorik Değişken Analizi

In [ ]:
def cat_analysis(df,col_name,info=True):
    print(f"{col_name.capitalize()} adlı değişkenin gözlemlerinin yüzdesel oranı:")
    ratio=(df[col_name].value_counts()/len(df)*100)
    print(f"{ratio}")
    if info:
        print("info".center(50,"#"))
        print(df.groupby(col_name).mean())
        print("\n\n")

In [ ]:
#Kategorik değişkenleri anlama,diğer değişkenler ile ilişkisi
for col in cat_cols:
    cat_analysis(df,col)

## 3. Veri Manipülasyonu

Veriler her zaman istediğimiz şekilde olmayabilir.Verileri istediğimiz,kullanabileceğimiz formata çevirmeye çalışacağız.

Örneğin "age" değişkeni taşıdığı bilgi nedeniyle sınıflandırma için işimize yarayabilir.Ancak taşıdığı eşsiz değer sayısı çok fazla bunu azaltmamız lazım. 

In [ ]:
#"age" değişkeni 47 farklı değer taşıyor.
print(df["age"].nunique())

In [ ]:
df[num_cols].describe().T

In [ ]:
# "age" değişkeni belirtilen yaş aralıklarına göre sınıflandırılıyor.

df["age_cat"]=pd.cut(df["age"],[0,22,30,45,70],labels=["0_22","22_30","30_45","45_70"])

In [ ]:
def gg(df,target):
    col_names=[col for col in df.columns if col not in ["charges","age","bmi"]]
    #col_names=df.columns[~df.columns.str.contains("charges")]
    print(col_names)
    for col in col_names:
        
        ss=df.groupby(col)[target].mean()
        plt.ylabel(target)
        plt.title(f"{col} adlı değişken kırılımında ortalama ücret")
        plt.legend()
        #sns.set(rc={'figure.figsize': (8, 10)})
        sns.barplot(x=ss.index,y=ss.values)
        plt.show()
        print("\n")
        print(ss)
        print("######################\n\n")
    
        

In [ ]:
#yeni df için ortalama ücretlerin grafik üzerinde gösterilmesi
gg(df,"charges")

Persona sınıflandırmamızı "smoker","region","children","age_cat" değişkenlerine göre oluşturacağız.Bu yüzden bu değişkenlere göre kırılımların ücret ortalamalarını incelemekte fayda var.

In [ ]:
agg_df=df.groupby(["smoker","region","children","age_cat"]).agg({"charges":"mean"}).sort_values("charges",ascending=False)

In [ ]:
print(agg_df)

Burada görülüyor ki oluşturduğumuz senaryoda bazı değerler için elimizde hiç örnek yok.Bu örnekler için değer ataması yapmamız gerekiyor.

In [ ]:
agg_df=agg_df.reset_index()
print(agg_df.shape)

In [ ]:
def compare_draw(df1,col1,target,df2=False,col2=False):
    plt.subplot(1,2,1)
    ss=df1.groupby(col1)[target].mean()
    plt.ylabel(col1)
    plt.title(f"{col1} adlı değişken kırılımında ortalama ücret")
    plt.legend()
    sns.barplot(x=ss.index,y=ss.values)
    plt.xticks(rotation=45)
    plt.show()
    print("\n")
    print(ss)
    print("######################\n\n")
    if type(df2) != bool :
        plt.subplot(1,2,2)
        ss=df2.groupby(col2)[target].mean()
        plt.ylabel(col2)
        plt.title(f"{col2} adlı değişken kırılımında ortalama ücret")
        plt.legend()
        plt.xticks(rotation=45)
        sns.barplot(x=ss.index,y=ss.values)
        plt.show()
        print("\n")
        print(ss)
        print("######################\n\n")


In [ ]:
# Eksik veriler için ayrı bir dataframe oluşturuldu.
missing_values=agg_df[agg_df["charges"].isnull()]

In [ ]:
print(missing_values.tail(10))

In [ ]:
# 44 adet eksik değerimiz var
print(missing_values.shape)

In [ ]:
compare_draw(df,"region","charges")

Eksik değerleri doldurmak için şöyle bir yöntem seçtim.Bölgelere ait ücret ortalaması grafiğini hatırlarsak "northwest" ile "southwest" bölgesi ve "northeast" ile "southeast" bölgesi birbirine benzer davranışlar sergiliyor.Bu yüzden eksik verileri doldurmak için bu bölgelerin birbirinde karşılık gelen koşullarından fayda sağladım.

In [ ]:
def missing_fill(df):
    ab=agg_df[agg_df["charges"].isnull()]
    for i in ab.index:
        if (ab["smoker"][i] == "no") and (ab["region"][i] == "northeast"):
            ab["charges"][i] = agg_df[(agg_df["smoker"] == "no") & (agg_df["region"]=="southeast") & (agg_df["age_cat"]==ab["age_cat"][i])]["charges"].mean()
        
        elif (ab["smoker"][i] == "no") and (ab["region"][i] == "southeast"):
              ab["charges"][i] = agg_df[(agg_df["smoker"] == "no") & (agg_df["region"]=="northeast") & (agg_df["age_cat"]==ab["age_cat"][i])]["charges"].mean()
        
        elif (ab["smoker"][i] == "no") and (ab["region"][i] == "northwest"):
              ab["charges"][i] = agg_df[(agg_df["smoker"] == "no") & (agg_df["region"]=="southwest") & (agg_df["age_cat"]==ab["age_cat"][i])]["charges"].mean()
        
        elif (ab["smoker"][i] == "no") and (ab["region"][i] == "southwest"):
            ab["charges"][i] = agg_df[(agg_df["smoker"] == "no") & (agg_df["region"]=="northwest") & (agg_df["age_cat"]==ab["age_cat"][i])]["charges"].mean()
        
        elif (ab["smoker"][i] == "yes") and (ab["region"][i] == "northeast"):
            ab["charges"][i] = agg_df[(agg_df["smoker"] == "yes") & (agg_df["region"]=="southeast") & (agg_df["age_cat"]==ab["age_cat"][i])]["charges"].mean()
        
        elif (ab["smoker"][i] == "yes") and (ab["region"][i] == "southeast"):
              ab["charges"][i] = agg_df[(agg_df["smoker"] == "yes") & (agg_df["region"]=="northeast") & (agg_df["age_cat"]==ab["age_cat"][i])]["charges"].mean()
        
        elif (ab["smoker"][i] == "yes") and (ab["region"][i] == "northwest"):
              ab["charges"][i] = agg_df[(agg_df["smoker"] == "yes") & (agg_df["region"]=="southwest") & (agg_df["age_cat"]==ab["age_cat"][i])]["charges"].mean()
        
        elif (ab["smoker"][i] == "yes") and (ab["region"][i] == "southwest"):
            ab["charges"][i] = agg_df[(agg_df["smoker"] == "yes") & (agg_df["region"]=="northwest") & (agg_df["age_cat"]==ab["age_cat"][i])]["charges"].mean()
        
        
    return ab


In [ ]:
# Eksik değerler için tahminlerimiz bir değişkene atandı.
al=missing_fill(agg_df)

In [ ]:
# Üzerinde çalıştığımız df içindeki eksik değerler yerine atandı.
agg_df.loc[al.index.values]=al

In [ ]:
control_df(agg_df)

In [ ]:
compare_draw(df,"age_cat","charges",agg_df,"age_cat")

Görüldüğü üzere yaş kategorilerine göre ücret dağılımını incelediğimizde eksik verileri tamamladığımız zaman bütün ortalamalar ücretler yükselmiştir.
Bu durumun nedeni için doldurduğumuz eksik verileri incelemekte fayda var.

In [ ]:
print(missing_values["smoker"].value_counts())

Görüldüğü üzere doldurduğumuz eksik verilerin çoğunluğu sigara içen kişilerden oluşuyordu.Sigara içen kişilerin ödediği ücret ortalaması içmeyenlere göre çok daha yüksek olduğu için genel ücret ortalaması artmış oldu.

Aşağıda görüldüğü üzere smoker değişkeninin kırılımında anlamlı bir değişiklik olmadı.Bu eksik verileri, genel veri setinin dağılımına uygun tamamladığımızı gösterir. 

In [ ]:
compare_draw(df,"smoker","charges",agg_df,"smoker")

## 4.Persona Oluşturulması

Bu kısımda verilerimiz persona dataframe şekline çeviriliecektir.

In [ ]:
zz=[]
for i in range(len(agg_df)):
    tl=agg_df["region"][i] + "_" + str(agg_df["children"][i]) + "_" + agg_df["smoker"][i] + "_" + agg_df["age_cat"][i]
    zz.append(tl.upper())

In [ ]:
persona_df=pd.concat([pd.DataFrame(zz),agg_df["charges"]],axis=1)
persona_df.columns=["level_based","charges"]

In [ ]:
print(persona_df.shape)

Her bir personadan birden fazla olması ihtimaline karşı her persona sınıfı için ortalama değerleri alınmalı.

In [ ]:
persona_df=persona_df.groupby("level_based").agg({"charges":"mean"}).sort_values("charges",ascending=False)
persona_df=persona_df.reset_index()
persona_df.shape

Oluşturduğumuz persona dataframe'ine göz atalım.

In [ ]:
print(persona_df.head())

Ücretlerimizide ["F","E","D","C","B","A"] (F en düşük A en yüksek) 6 segmente ayırıyoruz.

In [ ]:
persona_df["SEGMENT"]=pd.qcut(persona_df["charges"],6,["F","E","D","C","B","A"])
persona_df=persona_df.reset_index()

In [ ]:
print(persona_df.head(10))

## 5. Sonuç

In [ ]:
def user(region,child,smoker,age):
    if age<22:
        age="0_22"
    elif age>=22 and age<30:
        age="22_30"
    elif age>=30 and age<45:
        age="30_45"
    elif age>=45 and age<=70:
        age="45_70"
    else:
        print("0-70 yaş aralığında bir sayı giriniz")
    
    s=region.upper() + "_" + str(child).upper() + "_"+ smoker.upper() + "_" + age.upper()
    
    t=persona_df[persona_df["level_based"]==s]
    print(t)

    return 
    

In [ ]:
user("southwest",5,"no",60)